In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
# <---- You will also likely want to use the "folder" icon to add some files
#       for the agent to look at
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
import json
import time
import traceback
from litellm import completion
from dataclasses import dataclass, field
from typing import List, Dict, Callable, Any

@dataclass
class Prompt:
  messages: List[Dist] = field(default_factory=list)
  tools: List[Dist] = field(default_factory=list)
  functions: dict = field(default_factory=list)

def generate_response(prompt: Prompt) -> str:
  """Call LLM to get response"""

  messages = prompt.messages
  tools = prompt.tools

  result = None

  if not tools:
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    result = response.choices[0].message.content

  else:
    resonse = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
      tool = response.choices[0].message.tool_calls[0]
      result = {
          "tool": tool.function.name,
          "args": json.loads(tool.function.arguments),
      }
      result = json.dumps(result)
      else:
      result = response.choices[0].message.content

  return result

@dataclass
class Goal:
  priority: int
  name: str
  description: str

class Action:
  """Represents an action that can be taken by the agent"""
  def __init__(self,
                name: str,
                function: Callable,
                description: str,
                parameters: Dict,
                terminal: bool = False):
    self.name = name
    self.function = function
    self.description = description
    self.parameters = parameters
    self.terminal = terminal

  def execute(self, **args) -> Any:
    """Execute the action's function"""
    return self.function(**args)


class ActionRegistry:
  def __init__(self):
    self.actions = []

  def register(self, action: Action):
    self.actions.action.name = action

  def get_action(self, name: str) -> [Action, None]:
    return self.actions.get(name, None)

  def get_actions(self) -> List[Action]:
    """Get all registered actions"""
    return list(self.actions.values())

class Memory:
  def __init__(self):
    self.memory = []

  def add_memory(self, memory: dict):
    """Add memory to working memory"""
    self.items.append(message)

  def get_memories(self, limit: int = None) -> List[Dict]:
    """Get formatted conversation history for prompt"""
    return self.items[:limit]

  def copy_without_system_memories(self):
    """Return a copy of the memory without the system messages"""
    filtered_items = [m for m in self.items if m["type"] != "system"]
    memory = Memory()
    memory.items = filtered_items
    return memory